# Análise Estatística: Efeitos do Estresse e Estratégia de Aprendizagem na Memória

**Projeto:** Investigação da interação entre estresse induzido (TSST-G) e estratégias de aprendizagem (Releitura vs. Prática de Lembrar) no desempenho da memória (Evocação Livre e Organização - ARC).

**Objetivo deste Notebook:** Realizar as análises estatísticas principais para testar as hipóteses do estudo, verificar a eficácia da manipulação experimental e explorar os dados, incorporando as melhores práticas discutidas e informações metodológicas.

**Versão dos Dados:** `measures_phd_valid.csv` (Dados preliminares)

In [ ]:
# 1. Importação de Bibliotecas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.sandbox.stats.multicomp import multipletests # Para correção de múltiplas comparações
import pingouin as pg # Para testes de pressupostos e tamanhos de efeito
import os

# Configurações de Visualização
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 12
sns.set_palette("viridis")

# Diretório para salvar outputs (gráficos, tabelas)
output_dir = "/home/ubuntu/projeto_analise_dados/analises_output_excel"
os.makedirs(output_dir, exist_ok=True)

# --- Importações adicionais para Excel ---

from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment
from math import sqrt

# --- Funções Auxiliares para Formatação APA e Excel (v5) ---
def get_degrees_freedom(model_or_df, source=None):
    """Helper to extract degrees of freedom robustly."""
    try:
        if isinstance(model_or_df, pd.DataFrame): # Pingouin ANOVA table
            if source:
                # Handle potential multi-index or different source names in 3-way ANOVA
                if source in model_or_df["Source"].tolist():
                    df_effect = model_or_df.loc[model_or_df["Source"] == source, "DF"].iloc[0]
                else:
                    df_effect = "err" # Source not found
                if "Residual" in model_or_df["Source"].tolist():
                     df_error = model_or_df.loc[model_or_df["Source"] == "Residual", "DF"].iloc[0]
                else:
                     df_error = "err" # Residual not found
                return int(df_effect) if isinstance(df_effect, (int, float)) else df_effect, \
                       int(df_error) if isinstance(df_error, (int, float)) else df_error
            else: # Assume first row is effect, last is residual if source not specified (less robust)
                df_effect = model_or_df["DF"].iloc[0]
                df_error = model_or_df["DF"].iloc[-1]
                return int(df_effect), int(df_error)
        elif hasattr(model_or_df, "df_num") and hasattr(model_or_df, "df_den"): # Statsmodels fit results
            return int(model_or_df.df_num), int(model_or_df.df_den)
        elif hasattr(model_or_df, "DF"): # Pingouin t-test table
             return int(model_or_df["DF"].iloc[0])
    except (IndexError, TypeError, KeyError):
        return "err", "err" # Return error string if extraction fails
    return "err", "err"

def format_p_value(p):
    """Formats p-value according to APA style."""
    if pd.isna(p):
        return "p = n.s."
    p = float(p)
    if p < 0.001:
        return "p < .001"
    elif p < 0.01:
        return f"p = {p:.3f}"[1:] # Remove leading zero
    elif p < 0.05:
        return f"p = {p:.3f}"[1:]
    else:
        return f"p = {p:.3f}"[1:]

def format_stat(value):
    """Formats F or t value."""
    if pd.isna(value):
        return "N/A"
    return f"{float(value):.2f}"

def format_eta_squared(value):
    """Formats eta-squared."""
    if pd.isna(value):
        return "N/A"
    if float(value) < 0.001:
         return "η²p < .001"
    return f"η²p = {float(value):.3f}"[1:]

def format_cohen_d(value):
    """Formats Cohen's d."""
    if pd.isna(value):
        return "N/A"
    return f"d = {float(value):.2f}"

def format_anova_apa_v5(aov_table, dv_name):
    """Formats 2x2x2 ANOVA results in APA style (v5)."""
    if aov_table is None or aov_table.empty:
        return f"Análise ANOVA para {dv_name} não pôde ser realizada."
        
    results = []
    # Define the order of effects for reporting (main effects, 2-way, 3-way)
    source_order = [
        "Group", "Strategy", "nback_performance", 
        "Group * Strategy", "Group * nback_performance", "Strategy * nback_performance",
        "Group * Strategy * nback_performance"
    ]
    
    # Filter sources present in the table and maintain order
    sources_in_table = [s for s in source_order if s in aov_table["Source"].tolist()]
    
    text = f"Uma ANOVA fatorial 2 (Grupo: Controle, Estressado) x 2 (Estratégia: Releitura, Prática de Lembrar) x 2 (Desempenho N-Back: Baixo, Alto) foi conduzida para examinar os efeitos na variável {dv_name}. "
    
    for source in sources_in_table:
        row = aov_table[aov_table["Source"] == source]
        if row.empty:
            continue
        
        f_val = row["F"].iloc[0]
        p_val = row["p-unc"].iloc[0]
        eta_sq = row["n2p"].iloc[0]
        df_effect, df_error = get_degrees_freedom(aov_table, source)
        
        # Describe the effect type
        if "*" in source:
            effect_desc = f"a interação {source.replace('nback_performance', 'N-Back')}"
        else:
            effect_desc = f"o efeito principal de {source.replace('nback_performance', 'N-Back')}"
        significance = "não foi significativo" if p_val >= 0.05 else "foi significativo"
        
        # Handle potential errors in df extraction
        if df_effect == "err" or df_error == "err":
            f_string = f"*F*(gl_efeito?, gl_erro?) = {format_stat(f_val)}"
        else:
            f_string = f"*F*({df_effect}, {df_error}) = {format_stat(f_val)}"
            
        results.append(f"{effect_desc} {significance}, {f_string}, {format_p_value(p_val)}, {format_eta_squared(eta_sq)}")
        
    # Join the results grammatically
    if len(results) == 0:
        text += "Nenhum efeito pôde ser analisado."
    elif len(results) == 1:
        text += results[0] + "."
    else:
        text += ", ".join(results[:-1]) + " e " + results[-1] + "."
        
    return text

def format_ttest_apa(ttest_table, comparison_desc, group1_desc=None, group2_desc=None, m1=None, sd1=None, m2=None, sd2=None):
    """Formats t-test results in APA style."""
    if ttest_table is None or ttest_table.empty:
        return f"Teste t para {comparison_desc} não pôde ser realizado."
        
    row = ttest_table.iloc[0]
    t_val = row["T"]
    p_val = row["p-val"]
    d_val = row["cohen-d"]
    df_val = get_degrees_freedom(ttest_table)
    test_type = "pareado" if row["paired"] else "de amostras independentes"
    alternative = row["alternative"]
    tail = "unilateral" if alternative != "two-sided" else "bilateral"
    
    significance = "não foi significativa" if p_val >= 0.05 else "foi significativa"
    
    text = f"Um teste t {test_type} ({tail}) indicou que a diferença {comparison_desc} {significance}, *t*({df_val}) = {format_stat(t_val)}, {format_p_value(p_val)}, {format_cohen_d(d_val)}."
    
    # Add means and SDs if provided
    if test_type == "de amostras independentes" and m1 is not None and sd1 is not None and m2 is not None and sd2 is not None and group1_desc and group2_desc:
        comparison_word = "similares"
        if significance == "foi significativa":
            comparison_word = "maiores" if m1 > m2 else "menores"
        text += f" O grupo {group1_desc} (*M* = {m1:.2f}, *DP* = {sd1:.2f}) apresentou escores {comparison_word} em comparação ao grupo {group2_desc} (*M* = {m2:.2f}, *DP* = {sd2:.2f})."
    elif test_type == "pareado" and m1 is not None and sd1 is not None and m2 is not None and sd2 is not None:
         text += f" Os escores mudaram de (*M* = {m1:.2f}, *DP* = {sd1:.2f}) para (*M* = {m2:.2f}, *DP* = {sd2:.2f})."
         
    return text

def format_mediation_apa(mediation_results):
    """Formats mediation analysis results in APA style."""
    if mediation_results is None or mediation_results.empty:
        return "Análise de mediação não pôde ser realizada."

    try:
        # Extract relevant paths and values
        indirect_effect = mediation_results.loc[mediation_results["path"] == "Indirect", "coef"].iloc[0]
        indirect_pval = mediation_results.loc[mediation_results["path"] == "Indirect", "pval"].iloc[0]
        indirect_ci_lower = mediation_results.loc[mediation_results["path"] == "Indirect", "CI[2.5%]"].iloc[0]
        indirect_ci_upper = mediation_results.loc[mediation_results["path"] == "Indirect", "CI[97.5%]"].iloc[0]
        
        direct_effect = mediation_results.loc[mediation_results["path"] == "X -> Y", "coef"].iloc[0]
        direct_pval = mediation_results.loc[mediation_results["path"] == "X -> Y", "pval"].iloc[0]
        
        total_effect = mediation_results.loc[mediation_results["path"] == "Total", "coef"].iloc[0]
        total_pval = mediation_results.loc[mediation_results["path"] == "Total", "pval"].iloc[0]
        
        x_m_effect = mediation_results.loc[mediation_results["path"] == "X -> M", "coef"].iloc[0]
        x_m_pval = mediation_results.loc[mediation_results["path"] == "X -> M", "pval"].iloc[0]
        
        m_y_effect = mediation_results.loc[mediation_results["path"] == "M -> Y", "coef"].iloc[0]
        m_y_pval = mediation_results.loc[mediation_results["path"] == "M -> Y", "pval"].iloc[0]

        # Build the APA text
        text = "Uma análise de mediação foi conduzida para investigar se o efeito da Estratégia (X) na Recordação Livre (Y) é mediado pela Organização Semântica (ARC, M), controlando pelo Grupo (covariável). "
        
        # Path X -> M
        significance_xm = "não foi significativo" if x_m_pval >= 0.05 else "foi significativo"
        text += f"O efeito da Estratégia no ARC {significance_xm} (β = {x_m_effect:.3f}, {format_p_value(x_m_pval)}). "
        
        # Path M -> Y
        significance_my = "não foi significativo" if m_y_pval >= 0.05 else "foi significativo"
        text += f"O efeito do ARC na Recordação Livre, controlando pela Estratégia e Grupo, {significance_my} (β = {m_y_effect:.3f}, {format_p_value(m_y_pval)}). "
        
        # Indirect Effect (Mediation)
        significance_indirect = "não foi significativo" if indirect_pval >= 0.05 else "foi significativo"
        text += f"O efeito indireto da Estratégia na Recordação Livre através do ARC {significance_indirect} (Efeito Indireto = {indirect_effect:.3f}, IC 95% [{indirect_ci_lower:.3f}, {indirect_ci_upper:.3f}], {format_p_value(indirect_pval)}). "
        
        # Direct Effect
        significance_direct = "não foi significativo" if direct_pval >= 0.05 else "foi significativo"
        text += f"O efeito direto da Estratégia na Recordação Livre, controlando pelo ARC e Grupo, {significance_direct} (β = {direct_effect:.3f}, {format_p_value(direct_pval)}). "
        
        # Total Effect
        significance_total = "não foi significativo" if total_pval >= 0.05 else "foi significativo"
        text += f"O efeito total da Estratégia na Recordação Livre {significance_total} (β = {total_effect:.3f}, {format_p_value(total_pval)})."
        
    except (IndexError, KeyError, TypeError) as e:
        return f"Erro ao formatar resultados da mediação: {e}. Verifique a tabela de resultados."
        
    return text

def add_analysis_to_excel(writer, sheet_name, df_results, intro_text, apa_text, img_path=None):
    """Adds analysis results (intro, table, APA text, image) to an Excel sheet."""
    workbook = writer.book
    try:
        # Remove default sheet if it exists and is empty
        if "Sheet1" in workbook.sheetnames and len(workbook["Sheet1"]._cells) == 0:
             del workbook["Sheet1"]
    except KeyError:
        pass # Sheet1 might not exist
        
    # Check if sheet already exists (relevant for append mode)
    if sheet_name in workbook.sheetnames:
        worksheet = workbook[sheet_name]
        # Clear existing content? Or append? For now, let's overwrite for simplicity if sheet exists.
        # To truly append, we'd need to find the last row.
        # Let's assume we overwrite if the sheet exists from a previous run of *this* function call.
        # A better approach might be needed if multiple calls write to the same sheet.
        print(f"Aviso: Planilha \'{sheet_name}\' já existe. Sobrescrevendo conteúdo.")
        # Clear existing content (simplistic approach)
        for row in worksheet.iter_rows():
            for cell in row:
                cell.value = None
        # Remove existing images? This is harder with openpyxl.
    else:
        worksheet = workbook.create_sheet(sheet_name)
    
    # Write Intro text first, merged cells
    current_row = 1
    if intro_text:
        worksheet.cell(row=current_row, column=1, value=intro_text)
        worksheet.merge_cells(start_row=current_row, start_column=1, end_row=current_row + 2, end_column=10) # Merge 3 rows for intro
        worksheet.cell(row=current_row, column=1).alignment = Alignment(wrap_text=True, vertical="top")
        worksheet.cell(row=current_row, column=1).font = Font(size=11, italic=True)
        current_row += 4 # Add a blank row after intro
        
    # Write APA text next, merged cells
    if apa_text:
        worksheet.cell(row=current_row, column=1, value=apa_text)
        worksheet.merge_cells(start_row=current_row, start_column=1, end_row=current_row + 2, end_column=10) # Merge 3 rows for APA
        worksheet.cell(row=current_row, column=1).alignment = Alignment(wrap_text=True, vertical="top")
        worksheet.cell(row=current_row, column=1).font = Font(size=11)
        current_row += 4 # Add a blank row after APA text
    
    # Write DataFrame results below the text
    start_row_df = current_row 
    if df_results is not None and not df_results.empty:
        # Convert index name if it's numeric or default, otherwise keep it
        df_to_write = df_results.copy()
        if df_to_write.index.name is None or isinstance(df_to_write.index.name, int):
             df_to_write.index.name = "Index"
        elif df_to_write.index.name == "path": # Specific for mediation
             df_to_write.index.name = "Caminho"
             
        for r_idx, row in enumerate(dataframe_to_rows(df_to_write, index=True, header=True), start_row_df):
            for c_idx, value in enumerate(row, 1):
                cell = worksheet.cell(row=r_idx, column=c_idx, value=value)
                # Apply formatting for numeric values (optional)
                if isinstance(value, (int, float)):
                    cell.number_format = '0.000' # Example format
                if r_idx == start_row_df: # Header row
                    cell.font = Font(bold=True)
        current_row += len(df_to_write) + 2 # Update current row after dataframe + blank row
    else:
        worksheet.cell(row=start_row_df, column=1, value="(Nenhuma tabela de dados para esta análise)")
        current_row += 2
                
    # Add image if path is provided
    img_anchor_row = current_row
    if img_path and os.path.exists(img_path):
        try:
            img = Image(img_path)
            # Optional: Resize image if needed
            # img.height = img.height * 0.5 
            # img.width = img.width * 0.5
            worksheet.add_image(img, f"A{img_anchor_row}") 
        except Exception as e:
            print(f"Erro ao adicionar imagem {img_path} à planilha {sheet_name}: {e}")
            worksheet.cell(row=img_anchor_row, column=1, value=f"(Erro ao carregar imagem: {os.path.basename(img_path)})" )
    elif img_path:
        print(f"Aviso: Arquivo de imagem não encontrado para adicionar à planilha {sheet_name}: {img_path}")
        worksheet.cell(row=img_anchor_row, column=1, value=f"(Imagem não encontrada: {os.path.basename(img_path)})" )



## 2. Carregamento e Inspeção Inicial dos Dados

Carregamos o conjunto de dados e realizamos uma inspeção inicial para verificar a estrutura, tipos de dados e primeiras linhas.

In [ ]:
# Carregar os dados
data_path = "/home/ubuntu/upload/measures_phd_valid.csv"
df = pd.read_csv(data_path)

# Exibir informações básicas
print("Dimensões do DataFrame:", df.shape)
print("\nTipos de Dados:\n", df.dtypes)
print("\nPrimeiras 5 linhas:\n", df.head())
print("\nValores Ausentes:\n", df.isnull().sum())

# --- Categorização do Desempenho N-Back ---
nback_col = 'nback_score'
if nback_col in df.columns:
    median_nback = df[nback_col].median()
    df['nback_performance'] = df[nback_col].apply(lambda x: 'Alto' if x > median_nback else 'Baixo')
    print(f"Coluna 'nback_performance' criada com base na mediana ({median_nback:.2f}) de '{nback_col}'.")
    print(df['nback_performance'].value_counts())
else:
    print(f"Aviso: Coluna '{nback_col}' não encontrada. A ANOVA 2x2x2 não pode ser realizada.")
    df['nback_performance'] = 'N/A' # Add placeholder column

## 3. Preparação e Limpeza dos Dados

Nesta seção, preparamos os dados para análise:
*   Selecionamos as colunas relevantes.
*   Renomeamos colunas para clareza, se necessário.
*   **Verificamos e/ou recalculamos variáveis importantes (ex: PANAS Negativo, BAIdelta)**, conforme discussão anterior.
*   Criamos rótulos descritivos para variáveis categóricas (Grupo, Estratégia).
*   Tratamos valores ausentes (se houver e for necessário).

In [ ]:
# Selecionar colunas relevantes (ajustar conforme necessidade)
relevant_cols = ["code", "Group", "Strategy", "words_recalled", "ARC", 
                 "BAIpre", "BAIpos", "BAIdelta", 
                 "PANASpre", "PANASpos", "PANASdelta", 
                 # Adicionar outras colunas se necessário (ex: para cálculo PANAS Negativo)
                ]
# df_analysis = df[relevant_cols].copy()

# !! Placeholder: Recalcular/Verificar PANAS Negativo !!
# Aqui entraria o código para calcular o PANAS Negativo, 
# garantindo que apenas os itens negativos sejam somados.
# Exemplo hipotético (precisa ser adaptado ao seu script original):
# panas_neg_items = [...] # Lista dos nomes das colunas dos itens negativos
# df_analysis["PANASneg_pre"] = df[panas_neg_items_pre].sum(axis=1)
# df_analysis["PANASneg_pos"] = df[panas_neg_items_pos].sum(axis=1)
# df_analysis["PANASneg_delta"] = df_analysis["PANASneg_pos"] - df_analysis["PANASneg_pre"]

# Criar rótulos descritivos
df["Group_label"] = df["Group"].map({0: "Controle", 1: "Estressado"})
df["Strategy_label"] = df["Strategy"].map({0: "Releitura", 1: "Prática de Lembrar"})

# Verificar novamente valores ausentes nas colunas selecionadas
# print("\nValores Ausentes (Após Seleção):\n", df_analysis.isnull().sum())

# Tratar ausentes (se necessário - exemplo: imputação ou remoção)
# df_analysis = df_analysis.dropna() # Exemplo: remover linhas com qualquer NA

# Exibir dataframe preparado
# print("\nDataFrame Preparado:\n", df_analysis.head())

## 4. Análise Descritiva Detalhada

Calculamos estatísticas descritivas (média, desvio padrão, N) e criamos visualizações (boxplots, histogramas) para as variáveis dependentes (ARC, `words_recalled`) e de manipulação (BAI, PANAS Negativo) por grupo e estratégia.

In [ ]:
# Estatísticas Descritivas
print("\n--- Estatísticas Descritivas ---")

# Exemplo para ARC
desc_arc = df.groupby(["Group_label", "Strategy_label"])["ARC"].agg(["count", "mean", "std"])
print("\nARC por Grupo e Estratégia:\n", desc_arc)

# Exemplo para Words Recalled
desc_words = df.groupby(["Group_label", "Strategy_label"])["words_recalled"].agg(["count", "mean", "std"])
print("\nPalavras Recordadas por Grupo e Estratégia:\n", desc_words)

# Descritivas para BAI e PANAS Negativo (pré, pós, delta) por Grupo
# ... (código similar para BAIpre, BAIpos, BAIdelta, PANASneg_pre, PANASneg_pos, PANASneg_delta)

# Visualizações
print("\n--- Gerando Visualizações Descritivas ---")

# Boxplot ARC
plt.figure()
sns.boxplot(x="Group_label", y="ARC", hue="Strategy_label", data=df)
plt.title("ARC por Grupo e Estratégia")
plt.xlabel("Grupo Experimental")
plt.ylabel("ARC Score")
plt.legend(title="Estratégia")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "boxplot_arc_grupo_estrategia.png"))
plt.show()

# Boxplot Words Recalled
plt.figure()
sns.boxplot(x="Group_label", y="words_recalled", hue="Strategy_label", data=df)
plt.title("Palavras Recordadas por Grupo e Estratégia")
plt.xlabel("Grupo Experimental")
plt.ylabel("Nº Palavras Recordadas")
plt.legend(title="Estratégia")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "boxplot_words_grupo_estrategia.png"))
plt.show()

# Boxplots para BAI e PANAS Negativo (pré/pós por grupo)
# ... (código para gerar boxplots comparando pré e pós para BAI e PANAS Negativo)

# --- Salvar Estatísticas Descritivas e Gráficos no Excel ---
excel_path = os.path.join(output_dir, 'resultados_analise_v5.xlsx')
intro_desc = "Esta seção apresenta as estatísticas descritivas (média e desvio padrão) para as principais variáveis dependentes (ARC, Palavras Recordadas, BAI, PANAS) divididas pelos grupos experimentais (Grupo, Estratégia). Os gráficos de barras e interação visualizam essas médias."
apa_desc = "As médias e desvios padrão para ARC, Palavras Recordadas, BAI e PANAS por Grupo e Estratégia são apresentadas nas tabelas. Os gráficos ilustram as tendências observadas."

# Save plots first to get paths
img_path_arc_bar, img_path_words_bar = None, None
img_path_arc_int, img_path_words_int = None, None
img_path_bai_bar, img_path_panas_bar = None, None
img_path_bai_int, img_path_panas_int = None, None
try:
    # ARC plots
    fig_arc_bar = plot_descriptive_bar(df, 'ARC', 'Grupo', 'Estratégia')
    img_path_arc_bar = os.path.join(output_dir, 'arc_barplot.png')
    fig_arc_bar.savefig(img_path_arc_bar)
    plt.close(fig_arc_bar)
    fig_arc_int = plot_descriptive_interaction(df, 'ARC', 'Grupo', 'Estratégia')
    img_path_arc_int = os.path.join(output_dir, 'arc_interacao.png')
    fig_arc_int.savefig(img_path_arc_int)
    plt.close(fig_arc_int)
    # Words plots
    fig_words_bar = plot_descriptive_bar(df, 'words_recalled', 'Grupo', 'Estratégia')
    img_path_words_bar = os.path.join(output_dir, 'palavras_barplot.png')
    fig_words_bar.savefig(img_path_words_bar)
    plt.close(fig_words_bar)
    fig_words_int = plot_descriptive_interaction(df, 'words_recalled', 'Grupo', 'Estratégia')
    img_path_words_int = os.path.join(output_dir, 'palavras_interacao.png')
    fig_words_int.savefig(img_path_words_int)
    plt.close(fig_words_int)
    # BAI plots
    fig_bai_bar = plot_descriptive_bar(df_long_stress, 'Score', 'Grupo', 'Tempo', y_label='BAI Score')
    img_path_bai_bar = os.path.join(output_dir, 'bai_barplot.png')
    fig_bai_bar.savefig(img_path_bai_bar)
    plt.close(fig_bai_bar)
    fig_bai_int = plot_descriptive_interaction(df_long_stress, 'Score', 'Grupo', 'Tempo', y_label='BAI Score')
    img_path_bai_int = os.path.join(output_dir, 'bai_interacao.png')
    fig_bai_int.savefig(img_path_bai_int)
    plt.close(fig_bai_int)
    # PANAS plots
    fig_panas_bar = plot_descriptive_bar(df_long_stress, 'Score', 'Grupo', 'Tempo', y_label='PANAS Negativo Score')
    img_path_panas_bar = os.path.join(output_dir, 'panas_barplot.png')
    fig_panas_bar.savefig(img_path_panas_bar)
    plt.close(fig_panas_bar)
    fig_panas_int = plot_descriptive_interaction(df_long_stress, 'Score', 'Grupo', 'Tempo', y_label='PANAS Negativo Score')
    img_path_panas_int = os.path.join(output_dir, 'panas_interacao.png')
    fig_panas_int.savefig(img_path_panas_int)
    plt.close(fig_panas_int)
except Exception as e:
    print(f"Erro ao gerar ou salvar gráficos descritivos: {e}")
    img_path_arc_bar, img_path_words_bar, img_path_arc_int, img_path_words_int = None, None, None, None
    img_path_bai_bar, img_path_panas_bar, img_path_bai_int, img_path_panas_int = None, None, None, None

# Write to Excel
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    # Write each descriptive table to its own sheet
    add_analysis_to_excel(writer, 'Desc_ARC', desc_arc, intro_desc, apa_desc, img_path_arc_int)
    add_analysis_to_excel(writer, 'Desc_Palavras', desc_words, intro_desc, apa_desc, img_path_words_int)
    add_analysis_to_excel(writer, 'Desc_BAI', desc_bai, intro_desc, apa_desc, img_path_bai_int)
    add_analysis_to_excel(writer, 'Desc_PANAS', desc_panas, intro_desc, apa_desc, img_path_panas_int)
print(f"Estatísticas descritivas adicionadas ao Excel: {excel_path}")

## 5. Verificação da Manipulação de Estresse

Verificamos se a manipulação de estresse (TSST-G) foi eficaz comparando as mudanças (delta) nas medidas de ansiedade (BAI) e afeto negativo (PANAS Negativo) entre os grupos Controle e Estressado. Também comparamos os níveis pré e pós dentro de cada grupo.

*   **Teste t pareado:** Compara pré vs. pós dentro de cada grupo.
*   **Teste t independente:** Compara os deltas (pós - pré) entre os grupos.

*Nota: Aplicaremos correção para múltiplas comparações posteriormente.*

In [ ]:
print("\n--- Verificação da Manipulação de Estresse ---")

# Separar dados por grupo
df_control = df[df["Group"] == 0]
df_stress = df[df["Group"] == 1]

# Testes t pareados (Pré vs. Pós)
bai_t_paired_control = pg.ttest(df_control["BAIpre"], df_control["BAIpos"], paired=True)
bai_t_paired_stress = pg.ttest(df_stress["BAIpre"], df_stress["BAIpos"], paired=True)
# panas_neg_t_paired_control = pg.ttest(df_control["PANASneg_pre"], df_control["PANASneg_pos"], paired=True)
# panas_neg_t_paired_stress = pg.ttest(df_stress["PANASneg_pre"], df_stress["PANASneg_pos"], paired=True)

print("\nTeste t Pareado - BAI Controle (Pré vs Pós):\n", bai_t_paired_control)
print("\nTeste t Pareado - BAI Estresse (Pré vs Pós):\n", bai_t_paired_stress)
# print("\nTeste t Pareado - PANAS Neg Controle (Pré vs Pós):\n", panas_neg_t_paired_control)
# print("\nTeste t Pareado - PANAS Neg Estresse (Pré vs Pós):\n", panas_neg_t_paired_stress)

# Testes t independentes (Comparando Deltas entre Grupos)
bai_t_ind_delta = pg.ttest(df_control["BAIdelta"], df_stress["BAIdelta"], correction=False) # Assumindo variâncias iguais inicialmente
# panas_neg_t_ind_delta = pg.ttest(df_control["PANASneg_delta"], df_stress["PANASneg_delta"], correction=False)

print("\nTeste t Independente - BAIdelta (Controle vs Estresse):\n", bai_t_ind_delta)
# print("\nTeste t Independente - PANASneg_delta (Controle vs Estresse):\n", panas_neg_t_ind_delta)

# Armazenar p-valores para correção posterior
p_values_manipulation = {
    "bai_paired_control": bai_t_paired_control["p-val"].iloc[0],
    "bai_paired_stress": bai_t_paired_stress["p-val"].iloc[0],
    # "panas_neg_paired_control": panas_neg_t_paired_control["p-val"].iloc[0],
    # "panas_neg_paired_stress": panas_neg_t_paired_stress["p-val"].iloc[0],
    "bai_ind_delta": bai_t_ind_delta["p-val"].iloc[0],
    # "panas_neg_ind_delta": panas_neg_t_ind_delta["p-val"].iloc[0]
}

# --- Salvar Verificação de Manipulação no Excel ---
excel_path = os.path.join(output_dir, 'resultados_analise_v5.xlsx')
intro_manip_bai = "Esta análise verifica se a manipulação de estresse (TSST-G vs. Controle) induziu diferenças nos níveis de ansiedade (BAI) entre os grupos, antes e depois da tarefa estressora. Um teste t é usado para comparar os escores BAI pré-tarefa entre os grupos, e outro para comparar os escores pós-tarefa."
intro_manip_panas = "Similarmente à análise do BAI, esta seção verifica se a manipulação de estresse afetou os escores de afeto negativo (PANAS Negativo). Testes t comparam os escores pré e pós-tarefa entre os grupos Controle e Estressado."
# Get means and SDs for APA text
bai_stats = df.groupby('Group')['BAIpre', 'BAIpos'].agg(['mean', 'std'])
panas_stats = df.groupby('Group')['PANASpre', 'PANASpost'].agg(['mean', 'std'])
apa_bai_full = ''
apa_panas_full = ''
if t_test_bai_pre is not None:
    m_c_pre, sd_c_pre = bai_stats.loc['Controle', ('BAIpre', 'mean')], bai_stats.loc['Controle', ('BAIpre', 'std')]
    m_s_pre, sd_s_pre = bai_stats.loc['Estressado', ('BAIpre', 'mean')], bai_stats.loc['Estressado', ('BAIpre', 'std')]
    apa_bai_pre = format_ttest_apa(t_test_bai_pre, 'nos níveis de BAI pré-tarefa entre os grupos', 'Controle', 'Estressado', m_c_pre, sd_c_pre, m_s_pre, sd_s_pre)
    apa_bai_full += apa_bai_pre + ' '
if t_test_bai_pos is not None:
    m_c_pos, sd_c_pos = bai_stats.loc['Controle', ('BAIpos', 'mean')], bai_stats.loc['Controle', ('BAIpos', 'std')]
    m_s_pos, sd_s_pos = bai_stats.loc['Estressado', ('BAIpos', 'mean')], bai_stats.loc['Estressado', ('BAIpos', 'std')]
    apa_bai_pos = format_ttest_apa(t_test_bai_pos, 'nos níveis de BAI pós-tarefa entre os grupos', 'Controle', 'Estressado', m_c_pos, sd_c_pos, m_s_pos, sd_s_pos)
    apa_bai_full += apa_bai_pos
if t_test_panas_pre is not None:
    m_c_pre, sd_c_pre = panas_stats.loc['Controle', ('PANASpre', 'mean')], panas_stats.loc['Controle', ('PANASpre', 'std')]
    m_s_pre, sd_s_pre = panas_stats.loc['Estressado', ('PANASpre', 'mean')], panas_stats.loc['Estressado', ('PANASpre', 'std')]
    apa_panas_pre = format_ttest_apa(t_test_panas_pre, 'nos níveis de PANAS Negativo pré-tarefa entre os grupos', 'Controle', 'Estressado', m_c_pre, sd_c_pre, m_s_pre, sd_s_pre)
    apa_panas_full += apa_panas_pre + ' '
if t_test_panas_pos is not None:
    m_c_pos, sd_c_pos = panas_stats.loc['Controle', ('PANASpost', 'mean')], panas_stats.loc['Controle', ('PANASpost', 'std')]
    m_s_pos, sd_s_pos = panas_stats.loc['Estressado', ('PANASpost', 'mean')], panas_stats.loc['Estressado', ('PANASpost', 'std')]
    apa_panas_pos = format_ttest_apa(t_test_panas_pos, 'nos níveis de PANAS Negativo pós-tarefa entre os grupos', 'Controle', 'Estressado', m_c_pos, sd_c_pos, m_s_pos, sd_s_pos)
    apa_panas_full += apa_panas_pos

# Combine result tables for Excel sheet
bai_results_combined = pd.concat([t_test_bai_pre.assign(Comparacao='BAI Pré'), t_test_bai_pos.assign(Comparacao='BAI Pós')], ignore_index=True) if t_test_bai_pre is not None and t_test_bai_pos is not None else (t_test_bai_pre.assign(Comparacao='BAI Pré') if t_test_bai_pre is not None else t_test_bai_pos.assign(Comparacao='BAI Pós'))
panas_results_combined = pd.concat([t_test_panas_pre.assign(Comparacao='PANAS Pré'), t_test_panas_pos.assign(Comparacao='PANAS Pós')], ignore_index=True) if t_test_panas_pre is not None and t_test_panas_pos is not None else (t_test_panas_pre.assign(Comparacao='PANAS Pré') if t_test_panas_pre is not None else t_test_panas_pos.assign(Comparacao='PANAS Pós'))

# Append to Excel
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    if bai_results_combined is not None:
        add_analysis_to_excel(writer, 'Manipulacao_BAI', bai_results_combined, intro_manip_bai, apa_bai_full)
    if panas_results_combined is not None:
        add_analysis_to_excel(writer, 'Manipulacao_PANAS', panas_results_combined, intro_manip_panas, apa_panas_full)
print(f"Resultados da verificação de manipulação adicionados ao Excel: {excel_path}")

## 6. Verificação dos Pressupostos Estatísticos (ANOVA)

Antes de realizar a ANOVA, verificamos seus principais pressupostos:
1.  **Normalidade dos Resíduos:** Os resíduos do modelo ANOVA devem seguir uma distribuição normal (Teste de Shapiro-Wilk).
2.  **Homogeneidade das Variâncias (Homocedasticidade):** As variâncias dos grupos devem ser aproximadamente iguais (Teste de Levene).

Realizaremos esses testes para as variáveis dependentes principais: ARC e `words_recalled`.

In [ ]:
print("\n--- Verificação dos Pressupostos da ANOVA ---")

# Pressupostos para ARC
print("\n** Verificando pressupostos para ARC **")
# Ajustar modelo ANOVA para obter resíduos
model_arc = ols("ARC ~ C(Group) * C(Strategy)", data=df).fit()
residuals_arc = model_arc.resid

# 1. Normalidade dos Resíduos (Shapiro-Wilk)
shapiro_arc = stats.shapiro(residuals_arc)
print(f"Teste de Shapiro-Wilk (Normalidade dos Resíduos ARC): W={shapiro_arc.statistic:.4f}, p={shapiro_arc.pvalue:.4f}")
if shapiro_arc.pvalue < 0.05:
    print("-> Atenção: Resíduos para ARC podem não ser normalmente distribuídos.")
else:
    print("-> Resíduos para ARC parecem normalmente distribuídos.")

# 2. Homogeneidade das Variâncias (Levene)
levene_arc = pg.homoscedasticity(data=df, dv="ARC", group="Group", method="levene")
print(f"\nTeste de Levene (Homocedasticidade ARC por Grupo): W={levene_arc["W"].iloc[0]:.4f}, p={levene_arc["pval"].iloc[0]:.4f}")
if levene_arc["pval"].iloc[0] < 0.05:
    print("-> Atenção: Variâncias para ARC podem não ser homogêneas entre os Grupos.")
else:
    print("-> Variâncias para ARC parecem homogêneas entre os Grupos.")

levene_arc_strategy = pg.homoscedasticity(data=df, dv="ARC", group="Strategy", method="levene")
print(f"Teste de Levene (Homocedasticidade ARC por Estratégia): W={levene_arc_strategy["W"].iloc[0]:.4f}, p={levene_arc_strategy["pval"].iloc[0]:.4f}")
# ... (interpretação similar)

# Pressupostos para words_recalled
print("\n** Verificando pressupostos para Palavras Recordadas **")
# ... (código similar ao de ARC para words_recalled)

# Nota: Se os pressupostos forem violados, considerar transformações ou testes não paramétricos.

## 7. Análises Principais: ANOVA Fatorial 2x2

Realizamos ANOVAs fatoriais 2 (Grupo: Controle vs. Estresse) × 2 (Estratégia: Releitura vs. Prática de Lembrar) para as variáveis dependentes principais:
*   ARC (Adjusted Ratio of Clustering)
*   `words_recalled` (Número de Palavras Recordadas)

Reportamos os efeitos principais (Grupo, Estratégia) e o efeito de interação (Grupo × Estratégia), incluindo os tamanhos de efeito (Eta-squared parcial, η²p).

In [ ]:
# --- ANOVA Fatorial 2x2x2 (Grupo x Estratégia x N-Back) ---
import pingouin as pg

# ANOVA para ARC
aov_arc_2x2x2 = None
apa_arc_2x2x2 = 'ANOVA 2x2x2 para ARC não realizada (verificar coluna nback_performance).'
if 'nback_performance' in df.columns and df['nback_performance'].nunique() > 1:
    try:
        aov_arc_2x2x2 = pg.anova(data=df, dv='ARC', between=['Group', 'Strategy', 'nback_performance'], detailed=True)
        print("
ANOVA 2x2x2 para ARC:")
        print(aov_arc_2x2x2)
        apa_arc_2x2x2 = format_anova_apa_v5(aov_arc_2x2x2, 'ARC')
    except Exception as e:
        print(f"Erro ao calcular ANOVA 2x2x2 para ARC: {e}")
        apa_arc_2x2x2 = f"Erro ao calcular ANOVA 2x2x2 para ARC: {e}"
else:
    print("Coluna 'nback_performance' não é adequada para ANOVA 2x2x2.")

# ANOVA para Palavras Recordadas
aov_words_2x2x2 = None
apa_words_2x2x2 = 'ANOVA 2x2x2 para Palavras Recordadas não realizada (verificar coluna nback_performance).'
if 'nback_performance' in df.columns and df['nback_performance'].nunique() > 1:
    try:
        aov_words_2x2x2 = pg.anova(data=df, dv='words_recalled', between=['Group', 'Strategy', 'nback_performance'], detailed=True)
        print("
ANOVA 2x2x2 para Palavras Recordadas:")
        print(aov_words_2x2x2)
        apa_words_2x2x2 = format_anova_apa_v5(aov_words_2x2x2, 'Palavras Recordadas')
    except Exception as e:
        print(f"Erro ao calcular ANOVA 2x2x2 para Palavras Recordadas: {e}")
        apa_words_2x2x2 = f"Erro ao calcular ANOVA 2x2x2 para Palavras Recordadas: {e}"
else:
    print("Coluna 'nback_performance' não é adequada para ANOVA 2x2x2.")

# --- Salvar Resultados da ANOVA 2x2x2 no Excel ---
excel_path = os.path.join(output_dir, 'resultados_analise_v5.xlsx')
intro_anova = "Esta análise utiliza uma ANOVA fatorial 2x2x2 para investigar os efeitos principais e as interações entre Grupo (Controle vs. Estresse), Estratégia de Aprendizagem (Releitura vs. Prática de Lembrar) e Desempenho na Tarefa N-Back (Baixo vs. Alto) nas medidas de memória (ARC e Palavras Recordadas). Permite verificar se o desempenho cognitivo basal (N-Back) modula os efeitos do estresse e da estratégia na memória."
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    if aov_arc_2x2x2 is not None:
        add_analysis_to_excel(writer, 'ANOVA_ARC_2x2x2', aov_arc_2x2x2, intro_anova, apa_arc_2x2x2)
    if aov_words_2x2x2 is not None:
        add_analysis_to_excel(writer, 'ANOVA_Words_2x2x2', aov_words_2x2x2, intro_anova, apa_words_2x2x2)
print(f"Resultados da ANOVA 2x2x2 adicionados ao Excel: {excel_path}")

## 8. Análise da Hipótese e Efeitos de Interação

Analisamos especificamente a hipótese principal: "o estresse diminui os scores de evocação total e ARC, e essa diminuição será menor ou inexistente no grupo que usou Prática de Lembrar".

Isso envolve examinar o efeito de interação. Como a interação não foi significativa nas análises preliminares (e seguindo a instrução do usuário), **enfatizaremos a ausência de evidência estatística para a interação**.

Realizaremos comparações planejadas (testes t) para explorar os efeitos simples (efeito do estresse dentro de cada estratégia), aplicando correção para múltiplas comparações.

In [ ]:
print("\n--- Análise da Hipótese e Efeitos de Interação ---")

# Interpretação da Interação (com base nos resultados da ANOVA)
interaction_p_arc = anova_arc.loc[anova_arc["Source"] == "Group * Strategy", "p-unc"].iloc[0]
interaction_p_words = anova_words.loc[anova_words["Source"] == "Group * Strategy", "p-unc"].iloc[0]

print(f"\nInteração Grupo x Estratégia para ARC: p = {interaction_p_arc:.4f}")
print(f"Interação Grupo x Estratégia para Palavras Recordadas: p = {interaction_p_words:.4f}")

if interaction_p_arc >= 0.05:
    print("-> Não há evidência estatística de interação significativa para ARC.")
if interaction_p_words >= 0.05:
    print("-> Não há evidência estatística de interação significativa para Palavras Recordadas.")

# Comparações Planejadas (Efeito do Estresse dentro de cada Estratégia)
# Separar dados por estratégia
df_releitura = df[df["Strategy"] == 0]
df_pratica = df[df["Strategy"] == 1]

# Teste t: Controle vs Estresse (dentro da Releitura)
arc_t_releitura = pg.ttest(df_releitura[df_releitura["Group"] == 0]["ARC"], 
                             df_releitura[df_releitura["Group"] == 1]["ARC"], correction=False)
words_t_releitura = pg.ttest(df_releitura[df_releitura["Group"] == 0]["words_recalled"], 
                               df_releitura[df_releitura["Group"] == 1]["words_recalled"], correction=False)

# Teste t: Controle vs Estresse (dentro da Prática de Lembrar)
arc_t_pratica = pg.ttest(df_pratica[df_pratica["Group"] == 0]["ARC"], 
                           df_pratica[df_pratica["Group"] == 1]["ARC"], correction=False)
words_t_pratica = pg.ttest(df_pratica[df_pratica["Group"] == 0]["words_recalled"], 
                             df_pratica[df_pratica["Group"] == 1]["words_recalled"], correction=False)

print("\n-- Comparações Planejadas (Testes t) --")
print("\nARC - Efeito do Estresse (Releitura):\n", arc_t_releitura)
print("\nARC - Efeito do Estresse (Prática):\n", arc_t_pratica)
print("\nPalavras - Efeito do Estresse (Releitura):\n", words_t_releitura)
print("\nPalavras - Efeito do Estresse (Prática):\n", words_t_pratica)

# Coletar p-valores para correção
p_values_planned = {
    "arc_releitura": arc_t_releitura["p-val"].iloc[0],
    "arc_pratica": arc_t_pratica["p-val"].iloc[0],
    "words_releitura": words_t_releitura["p-val"].iloc[0],
    "words_pratica": words_t_pratica["p-val"].iloc[0]
}

# Combinar todos os p-valores que precisam de correção
all_p_values = {**p_values_manipulation, **p_values_planned}
p_list = list(all_p_values.values())
keys_list = list(all_p_values.keys())

# Aplicar correção (ex: Bonferroni ou FDR)
reject_bonferroni, pvals_corrected_bonf, _, _ = multipletests(p_list, alpha=0.05, method='bonferroni')
reject_fdr, pvals_corrected_fdr, _, _ = multipletests(p_list, alpha=0.05, method='fdr_bh')

print("\n-- Correção para Múltiplas Comparações --")
results_correction = pd.DataFrame({
    "Teste": keys_list,
    "p_original": p_list,
    "p_corrigido_Bonf": pvals_corrected_bonf,
    "Significativo_Bonf": reject_bonferroni,
    "p_corrigido_FDR": pvals_corrected_fdr,
    "Significativo_FDR": reject_fdr
})
print(results_correction)

print("\nInterpretar resultados com base nos p-valores corrigidos.")

# --- Salvar Comparações Planejadas no Excel ---
excel_path = os.path.join(output_dir, 'resultados_analise_v5.xlsx')
intro_planned = "Após a ANOVA, comparações planejadas (testes t) são realizadas para investigar hipóteses específicas sobre as diferenças entre condições particulares (e.g., comparar Prática de Lembrar vs. Releitura dentro do grupo Estressado). Isso permite um exame mais focado dos efeitos encontrados."
# Combine all planned comparison results into one DataFrame for the sheet
all_planned_results = []
apa_planned_full = ''
# ARC Comparisons
if comp_arc_rp_control is not None:
    all_planned_results.append(comp_arc_rp_control.assign(Comparacao='ARC: RP vs Rel (Controle)'))
    apa_planned_full += format_ttest_apa(comp_arc_rp_control, 'entre Prática de Lembrar e Releitura no ARC (Grupo Controle)') + ' '
if comp_arc_rp_stress is not None:
    all_planned_results.append(comp_arc_rp_stress.assign(Comparacao='ARC: RP vs Rel (Estresse)'))
    apa_planned_full += format_ttest_apa(comp_arc_rp_stress, 'entre Prática de Lembrar e Releitura no ARC (Grupo Estresse)') + ' '
if comp_arc_control_stress_rel is not None:
    all_planned_results.append(comp_arc_control_stress_rel.assign(Comparacao='ARC: Controle vs Estresse (Releitura)'))
    apa_planned_full += format_ttest_apa(comp_arc_control_stress_rel, 'entre Controle e Estresse no ARC (Estratégia Releitura)') + ' '
if comp_arc_control_stress_rp is not None:
    all_planned_results.append(comp_arc_control_stress_rp.assign(Comparacao='ARC: Controle vs Estresse (RP)'))
    apa_planned_full += format_ttest_apa(comp_arc_control_stress_rp, 'entre Controle e Estresse no ARC (Estratégia Prática de Lembrar)') + ' '
# Words Recalled Comparisons
if comp_words_rp_control is not None:
    all_planned_results.append(comp_words_rp_control.assign(Comparacao='Palavras: RP vs Rel (Controle)'))
    apa_planned_full += format_ttest_apa(comp_words_rp_control, 'entre Prática de Lembrar e Releitura em Palavras Recordadas (Grupo Controle)') + ' '
if comp_words_rp_stress is not None:
    all_planned_results.append(comp_words_rp_stress.assign(Comparacao='Palavras: RP vs Rel (Estresse)'))
    apa_planned_full += format_ttest_apa(comp_words_rp_stress, 'entre Prática de Lembrar e Releitura em Palavras Recordadas (Grupo Estresse)') + ' '
if comp_words_control_stress_rel is not None:
    all_planned_results.append(comp_words_control_stress_rel.assign(Comparacao='Palavras: Controle vs Estresse (Releitura)'))
    apa_planned_full += format_ttest_apa(comp_words_control_stress_rel, 'entre Controle e Estresse em Palavras Recordadas (Estratégia Releitura)') + ' '
if comp_words_control_stress_rp is not None:
    all_planned_results.append(comp_words_control_stress_rp.assign(Comparacao='Palavras: Controle vs Estresse (RP)'))
    apa_planned_full += format_ttest_apa(comp_words_control_stress_rp, 'entre Controle e Estresse em Palavras Recordadas (Estratégia Prática de Lembrar)') + ' '
# Combine results if any exist
if all_planned_results:
    planned_results_combined = pd.concat(all_planned_results, ignore_index=True)
    # Append to Excel
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        add_analysis_to_excel(writer, 'Comparacoes_Planejadas', planned_results_combined, intro_planned, apa_planned_full.strip())
    print(f"Comparações planejadas adicionadas ao Excel: {excel_path}")
else:
    print("Nenhuma comparação planejada foi realizada ou gerou resultados.")

## 9. Visualizações dos Resultados Principais

Criamos gráficos de interação para visualizar os padrões dos efeitos principais e da interação (mesmo que não significativa) para ARC e `words_recalled`.

In [ ]:
print("\n--- Visualizações dos Resultados Principais ---")

# Gráfico de Interação para ARC
plt.figure()
sns.pointplot(x="Group_label", y="ARC", hue="Strategy_label", data=df, 
              markers=["o", "s"], linestyles=["-", "--"], dodge=True, errorbar="se")
plt.title("Gráfico de Interação: ARC por Grupo e Estratégia")
plt.xlabel("Grupo Experimental")
plt.ylabel("ARC Score Médio (± Erro Padrão)")
plt.legend(title="Estratégia")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "interaction_plot_arc.png"))
plt.show()

# Gráfico de Interação para Words Recalled
plt.figure()
sns.pointplot(x="Group_label", y="words_recalled", hue="Strategy_label", data=df, 
              markers=["o", "s"], linestyles=["-", "--"], dodge=True, errorbar="se")
plt.title("Gráfico de Interação: Palavras Recordadas por Grupo e Estratégia")
plt.xlabel("Grupo Experimental")
plt.ylabel("Nº Palavras Recordadas Médio (± Erro Padrão)")
plt.legend(title="Estratégia")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "interaction_plot_words.png"))
plt.show()

# --- Salvar Correção de Comparações Múltiplas no Excel ---
excel_path = os.path.join(output_dir, 'resultados_analise_v5.xlsx')
intro_corr = "Dado que múltiplas comparações planejadas foram realizadas, é necessário corrigir os p-valores para controlar a taxa de erro Tipo I (falsos positivos). O método de Bonferroni-Holm é aplicado aqui aos p-valores das comparações planejadas."
# Create APA text summarizing the correction
apa_corr_full = 'Os p-valores das comparações planejadas foram ajustados usando a correção de Bonferroni-Holm para controlar a taxa de erro Tipo I. '
if 'p-corr' in planned_results_combined.columns:
    significant_after_corr = planned_results_combined[planned_results_combined['p-corr'] < 0.05]
    if not significant_after_corr.empty:
        apa_corr_full += 'As seguintes comparações permaneceram significativas após a correção: '
        apa_corr_full += ', '.join(significant_after_corr['Comparacao'].tolist()) + '.'
    else:
        apa_corr_full += 'Nenhuma comparação permaneceu significativa após a correção.'
else:
    apa_corr_full += 'A correção não pôde ser aplicada (verificar resultados anteriores).'
    planned_results_combined = pd.DataFrame({'Status': ['Correção não aplicada']}) # Placeholder df

# Append to Excel
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    add_analysis_to_excel(writer, 'Correcao_Multipla', planned_results_combined[['Comparacao', 'p-val', 'p-corr', 'significant']], intro_corr, apa_corr_full)
print(f"Resultados da correção de comparações múltiplas adicionados ao Excel: {excel_path}")

## 10. Análise de Outliers (Placeholder)

Esta seção é reservada para a análise e tratamento de outliers.

*   Identificar outliers (ex: Z-score > 3 ou critério baseado em IQR).
*   Analisar o impacto dos outliers nos resultados.
*   Decidir sobre o tratamento (manter, remover, transformar) e justificar.

*(A implementação será definida após discussão com o usuário).*

In [ ]:
# Placeholder para código de análise de outliers
print("\n--- Análise de Outliers (a ser definida) ---")

# Exemplo: Identificação via Z-score
# from scipy.stats import zscore
# numeric_cols = ["ARC", "words_recalled", "BAIdelta", "PANASneg_delta"]
# z_scores = df[numeric_cols].apply(zscore)
# outliers = (np.abs(z_scores) > 3).any(axis=1)
# print(f"Número de participantes identificados como outliers (Z > 3): {outliers.sum()}")
# print("IDs dos outliers:", df.loc[outliers, "code"].tolist())

# Próximos passos: Discutir critérios e tratamento.

## 11. Conclusão e Resumo dos Achados

Resumo dos principais resultados estatísticos, considerando os p-valores corrigidos e os tamanhos de efeito.

*   Eficácia da manipulação de estresse.
*   Efeitos principais de Grupo e Estratégia na memória (ARC e palavras recordadas).
*   Ausência de interação significativa Grupo × Estratégia.
*   Resultados das comparações planejadas (efeito do estresse dentro de cada estratégia).
*   Implicações e próximos passos.

In [ ]:
# Placeholder para resumo textual ou tabelas finais
print("\n--- Conclusão e Resumo dos Achados (a ser preenchido) ---")

# Exemplo: Criar tabela resumo
# summary_table = ...
# print(summary_table)

In [ ]:
# --- Análise de Mediação (Estratégia -> ARC -> Palavras Recordadas | Grupo) ---
import pingouin as pg

mediation_results = None
apa_mediation = 'Análise de mediação não realizada.'
try:
    mediation_results = pg.mediation_analysis(data=df, x='Strategy', m='ARC', y='words_recalled', covar=['Group'], n_boot=5000, seed=42)
    print("
Resultados da Análise de Mediação:")
    print(mediation_results)
    apa_mediation = format_mediation_apa(mediation_results)
except Exception as e:
    print(f"Erro ao calcular Análise de Mediação: {e}")
    apa_mediation = f"Erro ao calcular Análise de Mediação: {e}"

# --- Salvar Resultados da Mediação no Excel ---
excel_path = os.path.join(output_dir, 'resultados_analise_v5.xlsx')
intro_mediation = "Esta análise investiga se a relação entre a Estratégia de Aprendizagem e o número de Palavras Recordadas é explicada (mediada) pela Organização Semântica (ARC), controlando pelo efeito do Grupo Experimental (Estresse vs. Controle). Utiliza bootstrapping para estimar o efeito indireto."
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    if mediation_results is not None:
        add_analysis_to_excel(writer, 'Analise_Mediacao', mediation_results, intro_mediation, apa_mediation)
print(f"Resultados da Análise de Mediação adicionados ao Excel: {excel_path}")